In [ ]:
import ctypes

def get_gdi_handles_ctypes():
    """Returns the number of GDI handles using the standard ctypes library."""
    try:
        # Define constants from the Windows API
        GR_GDIOBJECTS = 0

        # Load necessary libraries
        user32 = ctypes.windll.user32
        kernel32 = ctypes.windll.kernel32

        # Get a handle to the current process
        process_handle = kernel32.GetCurrentProcess()

        # Call the GetGuiResources function to get the GDI count
        gdi_count = user32.GetGuiResources(process_handle, GR_GDIOBJECTS)

        return gdi_count
    except (AttributeError, NameError):
        # This will fail on non-Windows systems
        return -1

# Example of how to use it
print(f"Current GDI handles (using ctypes): {get_gdi_handles_ctypes()}")

In [ ]:
import os
os.getpid()

In [3]:
import time
import psutil
import ctypes

def get_pids_by_name(process_name):
    """Gets a list of all PIDs for a given process name."""
    pids = []
    # Iterate over all running processes
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Check if the process name contains the target name (case-insensitive)
            if process_name.lower() in proc.info['name'].lower():
                pids.append(proc.info['pid'])
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            # This can happen if a process terminates while iterating
            pass
    return pids

def getGDIcount(pid):
    """Calculates GDI handles for a given PID using the original ctypes method."""
    try:
        user32 = ctypes.windll.user32
        kernel32 = ctypes.windll.kernel32
        
        # Get a handle to the process with the necessary access rights
        # PROCESS_QUERY_INFORMATION = 0x0400
        process_handle = kernel32.OpenProcess(0x0400, 0, pid)
        if not process_handle:
            return -1 # Failed to open process (e.g., access denied)
            
        # Get the GDI object count for the process
        # GR_GDIOBJECTS = 0
        gdi_count = user32.GetGuiResources(process_handle, 0)
        
        # Always close the handle to prevent leaks
        kernel32.CloseHandle(process_handle)
        return gdi_count
    except Exception as e:
        print(f"Error getting GDI count for PID {pid}: {e}")
        return -1

def monitor_gdi_for_all_processes(process_name, interval=0):
    """Monitors GDI handles for all processes matching the given name."""
    print(f"Searching for all processes named '{process_name}'...")
    
    try:
        pids = get_pids_by_name(process_name)
        if not pids:
            print(f"No processes named '{process_name}' found.")
            return
            
        print(f"Found {len(pids)} process(es). Now monitoring GDI handles for PIDs: {pids}")
        print("Press Ctrl+C to stop...")
        
        while True:
            print(f"\n--- Snapshot at {time.ctime()} ---")
            total_gdi = 0
            
            # Check GDI count for each found process
            for pid in pids:
                gdi_count = getGDIcount(pid)
                
                if gdi_count >= 0:
                    total_gdi += gdi_count
                    print(f"  PID: {pid:<6} | GDI Count: {gdi_count}")
                else:
                    # Handle cases where getting the count failed
                    print(f"  PID: {pid:<6} | Could not get GDI count (process may have ended or access denied).")

            print(f"--------------------------------------")
            print(f"Total GDI Handles for '{process_name}': {total_gdi}")
            
            if interval > 0:
                time.sleep(interval)
            else:
                break
                
    except KeyboardInterrupt:
        print("\n\nMonitoring stopped by user.")
    except Exception as e:
        print(f"\nAn error occurred: {e}")


process_to_monitor = 'python.exe'
monitor_gdi_for_all_processes(process_to_monitor)

Searching for all processes named 'python.exe'...
Found 7 process(es). Now monitoring GDI handles for PIDs: [2156, 7332, 10164, 16876, 21832, 22764, 22856]
Press Ctrl+C to stop...

--- Snapshot at Tue Jul 15 16:18:13 2025 ---
  PID: 2156   | GDI Count: 0
  PID: 7332   | GDI Count: 0
  PID: 10164  | GDI Count: 0
  PID: 16876  | GDI Count: 0
  PID: 21832  | GDI Count: 0
  PID: 22764  | GDI Count: 0
  PID: 22856  | GDI Count: 0
--------------------------------------
Total GDI Handles for 'python.exe': 0
